In [ ]:
import sys
sys.path.append('../../utils')

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import random

from rulekit import RuleKit
from rulekit.classification import RuleClassifier
from rulekit.params import Measures
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

RuleKit.init()

In [2]:
clf_simple = RuleClassifier(induction_measure=Measures.Precision,
                        pruning_measure=Measures.Precision,
                        voting_measure=Measures.Precision,
                        discrete_set_conditions_enabled = False,
                        negated_conditions_enabled = False,
                        intervals_conditions_enabled = False,
                        numerical_attributes_conditions_enabled = False,
                        nominal_attributes_conditions_enabled = False,
                        inner_alternatives_enabled = False,
                        )
clf_complex = RuleClassifier(induction_measure=Measures.Precision,
                        pruning_measure=Measures.Precision,
                        voting_measure=Measures.Precision,
                        discrete_set_conditions_enabled = True,
                        negated_conditions_enabled = True,
                        intervals_conditions_enabled = True,
                        numerical_attributes_conditions_enabled = True,
                        nominal_attributes_conditions_enabled = True,
                        inner_alternatives_enabled = False,
)

clf_complex_with_inners = RuleClassifier(induction_measure=Measures.Precision,
                        pruning_measure=Measures.Precision,
                        voting_measure=Measures.Precision,
                        discrete_set_conditions_enabled = True,
                        negated_conditions_enabled = True,
                        intervals_conditions_enabled = True,
                        numerical_attributes_conditions_enabled = True,
                        nominal_attributes_conditions_enabled = True,
                        inner_alternatives_enabled = True,
)


# Numeryczne

## a E {v1, v2}

In [3]:
df = pd.DataFrame(columns=["a", "b", "class"], index = [i for i in range(0,100)])


a = [round(random.uniform(0, 9),2) for i in range(100)]
b = [round(random.uniform(0, 9),2) for i in range(100)]

df["a"] = a
df["b"] = b

df.loc[(df["a"] >= 3) & (df["a"] <= 7),"class"] = "1"
df = df.fillna("0")
df

,a,b,class
0,2.58,0.36,0
1,1.99,4.29,0
2,2.48,5.98,0
3,2.51,2.47,0
4,7.68,0.13,0
...,...,...,...
95,8.30,2.32,0
96,3.45,4.27,1
97,1.41,8.76,0
98,2.85,3.22,0


In [6]:
df.to_csv("synth_1_test.csv", index = False)

In [4]:
fig = px.scatter(df, x='a',
            y='b',
            color='class',
            height=560,
            width=560,
            )
fig.show()

In [5]:
X = df.drop(['class'], axis=1)
y = df['class']

clf_simple.fit(X, y)

print("Rules original")
for rule in clf_simple.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules")
clf_complex.fit(X, y)
for rule in clf_complex.model.rules:
    print(rule, rule.stats)


print("\n\nComplex rules + inners alternatives")
clf_complex_with_inners.fit(X, y)
for rule in clf_complex_with_inners.model.rules:
    print(rule, rule.stats)

Rules original
IF a = (-inf, 2.99) THEN class = {0} (p = 40.0, n = 0.0, P = 60.0, N = 40.0, weight = 1.0, pvalue = 3.049449370164677e-13)
IF a = <7.00, inf) THEN class = {0} (p = 20.0, n = 0.0, P = 60.0, N = 40.0, weight = 1.0, pvalue = 7.820848065952753e-06)
IF a = <2.99, 7.00) THEN class = {1} (p = 40.0, n = 0.0, P = 40.0, N = 60.0, weight = 1.0, pvalue = 7.274719675172474e-29)


Complex rules
IF a = (-inf, 2.99) THEN class = {0} (p = 40.0, n = 0.0, P = 60.0, N = 40.0, weight = 1.0, pvalue = 3.049449370164677e-13)
IF a != <1.30, 7.04) THEN class = {0} (p = 37.0, n = 0.0, P = 60.0, N = 40.0, weight = 1.0, pvalue = 6.8377574217170884e-12)
IF a = <2.99, 7.04) THEN class = {1} (p = 40.0, n = 0.0, P = 40.0, N = 60.0, weight = 1.0, pvalue = 7.274719675172474e-29)


Complex rules + inners alternatives
IF #(a = <7.00, inf) OR a != <0.75, 7.06) OR a != <0.75, 7.07) OR a != <0.75, 7.16) OR a = <0.31, 0.73)) THEN class = {0} (p = 60.0, n = 0.0, P = 60.0, N = 40.0, weight = 1.0, pvalue = 7.27471

## a !E {v1, v2}

In [6]:
df = pd.DataFrame(columns=["a", "b", "class"], index = [i for i in range(0,100)])


a = [round(random.uniform(0, 9),2) for i in range(100)]
b = [round(random.uniform(0, 9),2) for i in range(100)]

df["a"] = a
df["b"] = b

df.loc[(df["a"] >= 3) & (df["a"] <= 7),"class"] = "0"
df = df.fillna("1")
df

,a,b,class
0,7.90,2.62,1
1,5.06,3.22,0
2,6.43,2.61,0
3,5.50,3.41,0
4,5.77,6.59,0
...,...,...,...
95,1.56,6.64,1
96,2.68,2.26,1
97,4.55,6.46,0
98,7.45,6.29,1


In [15]:
fig = px.scatter(df, x='a',
            y='b',
            color='class',
            height=560,
            width=560,
            )
fig.show()

In [16]:
X = df.drop(['class'], axis=1)
y = df['class']

clf_simple.fit(X, y)

print("Rules original")
for rule in clf_simple.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules")
clf_complex.fit(X, y)
for rule in clf_complex.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules + inners alternatives")
clf_complex_with_inners.fit(X, y)
for rule in clf_complex_with_inners.model.rules:
    print(rule, rule.stats)

Rules original
IF a = (-inf, 3) THEN class = {0} (p = 39.0, n = 0.0, P = 57.0, N = 43.0, weight = 1.0, pvalue = 3.442735067232518e-14)
IF a = <6.92, inf) THEN class = {0} (p = 18.0, n = 0.0, P = 57.0, N = 43.0, weight = 1.0, pvalue = 1.0120021985984552e-05)
IF a = <3, 6.92) THEN class = {1} (p = 43.0, n = 0.0, P = 43.0, N = 57.0, weight = 1.0, pvalue = 2.6235334749066164e-29)


Complex rules
IF a = (-inf, 3) THEN class = {0} (p = 39.0, n = 0.0, P = 57.0, N = 43.0, weight = 1.0, pvalue = 3.442735067232518e-14)
IF a != <1.55, 7.01) THEN class = {0} (p = 35.0, n = 0.0, P = 57.0, N = 43.0, weight = 1.0, pvalue = 3.186424265097899e-12)
IF a = <3, 7.01) THEN class = {1} (p = 43.0, n = 0.0, P = 43.0, N = 57.0, weight = 1.0, pvalue = 2.6235334749066164e-29)


Complex rules + inners alternatives
IF #(a = <6.92, inf) OR a != <1.05, 7.26) OR a != <1.05, 7.57) OR a != <1.05, 7.67) OR a = <0.33, 1.02)) THEN class = {0} (p = 57.0, n = 0.0, P = 57.0, N = 43.0, weight = 1.0, pvalue = 2.623533474906616

## a<b

In [17]:
df = pd.DataFrame(columns=["a", "b", "class"], index = [i for i in range(0,100)])


a = [round(random.uniform(0, 9),2) for i in range(100)]
b = [round(random.uniform(0, 9),2) for i in range(100)]

df["a"] = a
df["b"] = b

df.loc[(df["a"] < df["b"]),"class"] = "1"
df = df.fillna("0")
df

,a,b,class
0,5.81,5.72,0
1,0.50,5.94,1
2,5.89,3.63,0
3,2.86,4.89,1
4,8.98,5.26,0
...,...,...,...
95,2.26,1.46,0
96,2.25,0.24,0
97,6.24,4.04,0
98,4.21,0.56,0


In [18]:
fig = px.scatter(df, x='a',
            y='b',
            color='class',
            height=560,
            width=560,
            )
fig.show()

In [19]:
X = df.drop(['class'], axis=1)
y = df['class']

clf_simple.fit(X, y)

print("Rules original")
for rule in clf_simple.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules")
clf_complex.fit(X, y)
for rule in clf_complex.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules + inners alternatives")
clf_complex_with_inners.fit(X, y)
for rule in clf_complex_with_inners.model.rules:
    print(rule, rule.stats)

Rules original
IF b = (-inf, 1.93) THEN class = {0} (p = 23.0, n = 0.0, P = 54.0, N = 46.0, weight = 1.0, pvalue = 4.3672558946068994e-08)
IF a = <1.48, inf) AND b = (-inf, 3.18) THEN class = {0} (p = 29.0, n = 0.0, P = 54.0, N = 46.0, weight = 1.0, pvalue = 1.3562260204958005e-10)
IF a = <8.31, inf) AND b = (-inf, 8.58) THEN class = {0} (p = 9.0, n = 0.0, P = 54.0, N = 46.0, weight = 1.0, pvalue = 0.002795629973443157)
IF a = <5.11, inf) AND b = (-inf, 7.15) THEN class = {0} (p = 35.0, n = 0.0, P = 54.0, N = 46.0, weight = 1.0, pvalue = 1.6770654026831021e-13)
IF a = (-inf, 0.91) THEN class = {1} (p = 14.0, n = 0.0, P = 46.0, N = 54.0, weight = 1.0, pvalue = 5.428697471936758e-06)
IF b = <8.34, inf) THEN class = {1} (p = 10.0, n = 0.0, P = 46.0, N = 54.0, weight = 1.0, pvalue = 0.00023548686008518932)
IF a = (-inf, 1.73) AND b = <1.23, inf) THEN class = {1} (p = 19.0, n = 0.0, P = 46.0, N = 54.0, weight = 1.0, pvalue = 3.1390337742684785e-08)
IF a = (-inf, 2.23) AND b = <1.76, inf) TH

## a>b

In [58]:
df = pd.DataFrame(columns=["a", "b", "class"], index = [i for i in range(0,100)])


a = [round(random.uniform(0, 9),2) for i in range(100)]
b = [round(random.uniform(0, 9),2) for i in range(100)]

df["a"] = a
df["b"] = b

df.loc[(df["a"] > df["b"]),"class"] = "1"
df = df.fillna("0")
df

,a,b,class
0,0.29,8.53,0
1,6.04,6.57,0
2,1.06,4.50,0
3,6.13,4.54,1
4,0.32,0.51,0
...,...,...,...
95,0.96,3.62,0
96,3.75,2.34,1
97,4.09,6.49,0
98,0.73,4.04,0


In [59]:
fig = px.scatter(df, x='a',
            y='b',
            color='class',
            height=560,
            width=560,
            )
fig.show()

In [60]:
X = df.drop(['class'], axis=1)
y = df['class']

clf_simple.fit(X, y)

print("Rules original")
for rule in clf_simple.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules")
clf_complex.fit(X, y)
for rule in clf_complex.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules + inners alternatives")
clf_complex_with_inners.fit(X, y)
for rule in clf_complex_with_inners.model.rules:
    print(rule, rule.stats)

Rules original
IF b = <8.02, inf) THEN class = {0} (p = 11.0, n = 0.0, P = 47.0, N = 53.0, weight = 1.0, pvalue = 0.00012297647137782074)
IF a = (-inf, 0.46) THEN class = {0} (p = 7.0, n = 0.0, P = 47.0, N = 53.0, weight = 1.0, pvalue = 0.003928862103712904)
IF a = (-inf, 2.55) AND b = <0.28, inf) THEN class = {0} (p = 23.0, n = 0.0, P = 47.0, N = 53.0, weight = 1.0, pvalue = 6.484466745368206e-10)
IF a = (-inf, 6.42) AND b = <5.81, inf) THEN class = {0} (p = 26.0, n = 0.0, P = 47.0, N = 53.0, weight = 1.0, pvalue = 1.7941983175154166e-11)
IF a = (-inf, 3.73) AND b = <3.27, inf) THEN class = {0} (p = 27.0, n = 0.0, P = 47.0, N = 53.0, weight = 1.0, pvalue = 5.091643874030218e-12)
IF a = <8.50, inf) THEN class = {1} (p = 7.0, n = 0.0, P = 53.0, N = 47.0, weight = 1.0, pvalue = 0.009629392130748654)
IF a = <7.29, inf) AND b = (-inf, 8.26) THEN class = {1} (p = 15.0, n = 0.0, P = 53.0, N = 47.0, weight = 1.0, pvalue = 2.4671924945363415e-05)
IF a = <6.09, inf) AND b = (-inf, 8.26) THEN cl

## a = b

In [23]:
df = pd.DataFrame(columns=["a", "b", "class"], index = [i for i in range(0,100)])


a = [round(random.uniform(0, 9),2) for i in range(100)]
b = [round(random.uniform(0, 9),2) for i in range(100)]
b[70:] = a[70:]
df["a"] = a
df["b"] = b

df.loc[(df["a"] == df["b"]),"class"] = "1"
df = df.fillna("0")
df

,a,b,class
0,8.01,8.62,0
1,7.93,2.99,0
2,6.62,8.72,0
3,2.51,1.79,0
4,7.46,6.77,0
...,...,...,...
95,4.26,4.26,1
96,1.92,1.92,1
97,8.46,8.46,1
98,3.69,3.69,1


In [24]:
fig = px.scatter(df, x='a',
            y='b',
            color='class',
            height=560,
            width=560,
            )
fig.show()

In [25]:
X = df.drop(['class'], axis=1)
y = df['class']

clf_simple.fit(X, y)

print("Rules original")
for rule in clf_simple.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules")
clf_complex.fit(X, y)
for rule in clf_complex.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules + inners alternatives")
clf_complex_with_inners.fit(X, y)
for rule in clf_complex_with_inners.model.rules:
    print(rule, rule.stats)

Rules original
IF a = (-inf, 0.87) AND b = <0.31, inf) THEN class = {0} (p = 9.0, n = 0.0, P = 70.0, N = 30.0, weight = 1.0, pvalue = 0.03418801445376973)
IF b = <0.28, 0.91) THEN class = {0} (p = 8.0, n = 0.0, P = 70.0, N = 30.0, weight = 1.0, pvalue = 0.05073060209269053)
IF a = <1.78, inf) AND b = (-inf, 1.90) THEN class = {0} (p = 15.0, n = 0.0, P = 70.0, N = 30.0, weight = 1.0, pvalue = 0.0028478923418785848)
IF a = <2.18, inf) AND b = (-inf, 3.68) THEN class = {0} (p = 28.0, n = 0.0, P = 70.0, N = 30.0, weight = 1.0, pvalue = 5.593956006088468e-06)
IF a = (-inf, 3.66) AND b = <2.26, inf) THEN class = {0} (p = 26.0, n = 0.0, P = 70.0, N = 30.0, weight = 1.0, pvalue = 1.5971749653747414e-05)
IF a = (-inf, 7.50) AND b = <6.63, inf) THEN class = {0} (p = 13.0, n = 0.0, P = 70.0, N = 30.0, weight = 1.0, pvalue = 0.006675416823914652)
IF a = (-inf, 7.50) AND b = <2.26, inf) THEN class = {0} (p = 42.0, n = 13.0, P = 70.0, N = 30.0, weight = 0.7636363636363637, pvalue = 0.094238342475823

# Nominalne

## a E {zbiór wartości}

In [26]:
df = pd.DataFrame(columns=["a", "b", "class"], index = [i for i in range(0,100)])


a = [random.randint(0, 3) for i in range(100)]
b = [round(random.uniform(0, 9),2) for i in range(100)]


df["a"] = a
df["b"] = b

df = df.replace({'a': {0: "low", 1: "medium", 2: "high", 3: "vhigh"}})
df.loc[(df["a"] == "low") | (df["a"] == "medium"),"class"] = "1"
df = df.fillna("0")
df

,a,b,class
0,medium,6.32,1
1,low,8.90,1
2,high,4.49,0
3,low,3.06,1
4,medium,2.02,1
...,...,...,...
95,medium,4.37,1
96,high,5.42,0
97,vhigh,1.30,0
98,high,1.73,0


In [27]:
fig = px.scatter(df, x='a',
            y='b',
            color='class',
            height=560,
            width=560,
            )
fig.show()

In [28]:
X = df.drop(['class'], axis=1)
y = df['class']

clf_simple.fit(X, y)

print("Rules original")
for rule in clf_simple.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules")
clf_complex.fit(X, y)
for rule in clf_complex.model.rules:
    print(rule, rule.stats)


print("\n\nComplex rules + inners alternatives")
clf_complex_with_inners.fit(X, y)
for rule in clf_complex_with_inners.model.rules:
    print(rule, rule.stats)

Rules original
IF a = {low} THEN class = {1} (p = 23.0, n = 0.0, P = 43.0, N = 57.0, weight = 1.0, pvalue = 3.8630865717087334e-11)
IF a = {medium} THEN class = {1} (p = 20.0, n = 0.0, P = 43.0, N = 57.0, weight = 1.0, pvalue = 1.7921580617255256e-09)
IF a = {vhigh} THEN class = {0} (p = 31.0, n = 0.0, P = 57.0, N = 43.0, weight = 1.0, pvalue = 1.8425865977266838e-10)
IF a = {high} THEN class = {0} (p = 26.0, n = 0.0, P = 57.0, N = 43.0, weight = 1.0, pvalue = 1.746902357842758e-08)


Complex rules
IF a = {medium, low} THEN class = {1} (p = 43.0, n = 0.0, P = 43.0, N = 57.0, weight = 1.0, pvalue = 2.6235334749066164e-29)
IF a = {high, vhigh} THEN class = {0} (p = 57.0, n = 0.0, P = 57.0, N = 43.0, weight = 1.0, pvalue = 2.6235334749066164e-29)


Complex rules + inners alternatives
IF a = {medium, low} THEN class = {1} (p = 43.0, n = 0.0, P = 43.0, N = 57.0, weight = 1.0, pvalue = 2.6235334749066164e-29)
IF a = {high, vhigh} THEN class = {0} (p = 57.0, n = 0.0, P = 57.0, N = 43.0, weigh

## a != wartość

In [55]:
df = pd.DataFrame(columns=["a", "b", "class"], index = [i for i in range(0,100)])


a = [random.randint(0, 2) for i in range(100)]
b = [round(random.uniform(0, 9),2) for i in range(100)]


df["a"] = a
df["b"] = b

df = df.replace({'a': {0: "low", 1: "medium", 2: "high"}})
df.loc[(df["a"] == "low") | (df["a"] == "medium"),"class"] = "1"
df = df.fillna("0")
df

,a,b,class
0,low,0.71,1
1,low,2.60,1
2,high,5.30,0
3,high,4.68,0
4,low,2.14,1
...,...,...,...
95,low,6.32,1
96,medium,6.81,1
97,low,0.96,1
98,high,1.43,0


In [56]:
fig = px.scatter(df, x='a',
            y='b',
            color='class',
            height=560,
            width=560,
            )
fig.show()

In [57]:
X = df.drop(['class'], axis=1)
y = df['class']

clf_simple.fit(X, y)

print("Rules original")
for rule in clf_simple.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules")
clf_complex.fit(X, y)
for rule in clf_complex.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules + inners alternatives")
clf_complex_with_inners.fit(X, y)
for rule in clf_complex_with_inners.model.rules:
    print(rule, rule.stats)

Rules original
IF a = {medium} THEN class = {1} (p = 36.0, n = 0.0, P = 62.0, N = 38.0, weight = 1.0, pvalue = 1.060939428469097e-10)
IF a = {low} THEN class = {1} (p = 26.0, n = 0.0, P = 62.0, N = 38.0, weight = 1.0, pvalue = 2.998527462493941e-07)
IF a = {high} THEN class = {0} (p = 38.0, n = 0.0, P = 38.0, N = 62.0, weight = 1.0, pvalue = 1.7636531930450345e-28)


Complex rules
IF a != {high} THEN class = {1} (p = 62.0, n = 0.0, P = 62.0, N = 38.0, weight = 1.0, pvalue = 1.7636531930450345e-28)
IF a = {high} THEN class = {0} (p = 38.0, n = 0.0, P = 38.0, N = 62.0, weight = 1.0, pvalue = 1.7636531930450345e-28)


Complex rules + inners alternatives
IF a != {high} THEN class = {1} (p = 62.0, n = 0.0, P = 62.0, N = 38.0, weight = 1.0, pvalue = 1.7636531930450345e-28)
IF a = {high} THEN class = {0} (p = 38.0, n = 0.0, P = 38.0, N = 62.0, weight = 1.0, pvalue = 1.7636531930450345e-28)


## a = b 

In [31]:
df = pd.DataFrame(columns=["a", "b", "class"], index = [i for i in range(0,100)])


a = [random.randint(0, 2) for i in range(100)]
b = [random.randint(0, 2) for i in range(100)]


df["a"] = a
df["b"] = b

df = df.replace({'a': {0: "low", 1: "medium", 2: "high"}, 'b': {0: "low", 1: "medium", 2: "high"}})
df.loc[(df["a"] == df["b"]),"class"] = "1"
df = df.fillna("0")
df

,a,b,class
0,medium,low,0
1,low,low,1
2,medium,low,0
3,medium,medium,1
4,medium,high,0
...,...,...,...
95,high,low,0
96,medium,medium,1
97,medium,low,0
98,low,medium,0


In [33]:
fig = px.scatter(df, x='a',
            y='b',
            color='class',
            height=560,
            width=560,
            )
fig.show()

In [34]:
X = df.drop(['class'], axis=1)
y = df['class']

clf_simple.fit(X, y)

print("Rules original")
for rule in clf_simple.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules")
clf_complex.fit(X, y)
for rule in clf_complex.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules + inners alternatives")
clf_complex_with_inners.fit(X, y)
for rule in clf_complex_with_inners.model.rules:
    print(rule, rule.stats)

Rules original
IF a = {low} AND b = {medium} THEN class = {0} (p = 14.0, n = 0.0, P = 69.0, N = 31.0, weight = 1.0, pvalue = 0.0034988391628794106)
IF a = {low} AND b = {high} THEN class = {0} (p = 9.0, n = 0.0, P = 69.0, N = 31.0, weight = 1.0, pvalue = 0.02979241259542801)
IF a = {medium} AND b = {low} THEN class = {0} (p = 14.0, n = 0.0, P = 69.0, N = 31.0, weight = 1.0, pvalue = 0.0034988391628794106)
IF a = {high} AND b = {low} THEN class = {0} (p = 8.0, n = 0.0, P = 69.0, N = 31.0, weight = 1.0, pvalue = 0.04493281899638327)
IF a = {medium} AND b = {high} THEN class = {0} (p = 17.0, n = 0.0, P = 69.0, N = 31.0, weight = 1.0, pvalue = 0.0008969322399662095)
IF a = {medium} AND b = {medium} THEN class = {1} (p = 17.0, n = 0.0, P = 31.0, N = 69.0, weight = 1.0, pvalue = 3.987626267238035e-11)
IF a = {high} AND b = {high} THEN class = {1} (p = 9.0, n = 0.0, P = 31.0, N = 69.0, weight = 1.0, pvalue = 1.0598116859877885e-05)


Complex rules
IF a != b THEN class = {0} (p = 69.0, n = 0.0

# XOR

In [35]:
df = pd.DataFrame(columns=["a", "b", "class"], index = [i for i in range(0,4)])


a = ["0","0","1","1"]
b = ["0","1","0","1"]

df["a"] = a
df["b"] = b

df.loc[((df["a"] == "0") & (df["b"] == "1")) | ((df["a"] == "1") & (df["b"] == "0")) ,"class"] = "1"
df = df.fillna("0")
df

,a,b,class
0,0,0,0
1,0,1,1
2,1,0,1
3,1,1,0


In [36]:
fig = px.scatter(df, x='a',
            y='b',
            color='class',
            height=560,
            width=560,
            )
fig.show()

In [37]:
X = df.drop(['class'], axis=1)
y = df['class']

clf_simple.fit(X, y)

print("Rules original")
for rule in clf_simple.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules")
clf_complex.fit(X, y)
for rule in clf_complex.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules + inners alternatives")
clf_complex_with_inners.fit(X, y)
for rule in clf_complex_with_inners.model.rules:
    print(rule, rule.stats)


Rules original


Complex rules
IF a = b THEN class = {0} (p = 2.0, n = 0.0, P = 2.0, N = 2.0, weight = 1.0, pvalue = 0.16666666666666669)
IF a != b THEN class = {1} (p = 2.0, n = 0.0, P = 2.0, N = 2.0, weight = 1.0, pvalue = 0.16666666666666669)


Complex rules + inners alternatives
IF a = b THEN class = {0} (p = 2.0, n = 0.0, P = 2.0, N = 2.0, weight = 1.0, pvalue = 0.16666666666666669)
IF a != b THEN class = {1} (p = 2.0, n = 0.0, P = 2.0, N = 2.0, weight = 1.0, pvalue = 0.16666666666666669)


# Parity problem 
AN EMPIRICAL INVESTIGATION INTO DEEP AND SHALLOW RULE LEARNING

Consider, e.g., the following example: the parity concept, which is known to be hard to learn for heuristic, greedy learning algorithms,
checks whether an odd or an even number of R relevant attributes (out of a possibly higher total number of attributes) are
set to true. Figure 1a shows a flat rule-based representation of the target concept for R = 5, which requires 2^R−1 = 16
rules. On the other hand, a structured representation, which introduces three auxiliary predicates (parity2345,
parity345 and parity45 as shown in Figure 1b), is much more concise using only 2 · (R − 1) = 8 rules.

In [38]:
df = pd.DataFrame(columns=["x1", "x2", "x3","x4", "x5", "class"], index = [i for i in range(0,100)])

df["x1"] = [random.randint(0, 1) for i in range(100)]
df["x2"] = [random.randint(0, 1) for i in range(100)]
df["x3"] = [random.randint(0, 1) for i in range(100)]
df["x4"] = [random.randint(0, 1) for i in range(100)]
df["x5"] = [random.randint(0, 1) for i in range(100)]

df.loc[(df[["x1", "x2", "x3","x4", "x5"]].sum(axis=1)%2) == 0, "class"] = "1"
df = df.fillna("0")
df = df.astype("str")
df

,x1,x2,x3,x4,x5,class
0,0,0,0,0,0,1
1,1,1,1,1,1,0
2,0,1,0,1,0,1
3,1,0,0,0,0,0
4,0,0,1,0,0,0
...,...,...,...,...,...,...
95,0,1,0,0,1,1
96,0,0,0,0,1,0
97,0,0,0,1,1,1
98,1,1,0,0,0,1


In [39]:
df.dtypes

x1       object
x2       object
x3       object
x4       object
x5       object
class    object
dtype: object

In [40]:
X = df.drop(['class'], axis=1)
y = df['class']

clf_simple.fit(X, y)

print("Rules original")
for rule in clf_simple.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules")
clf_complex.fit(X, y)
for rule in clf_complex.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules + inners alternatives")
clf_complex_with_inners.fit(X, y)
for rule in clf_complex_with_inners.model.rules:
    print(rule, rule.stats)

Rules original
IF x1 = {1} AND x2 = {0} AND x3 = {0} AND x4 = {1} AND x5 = {0} THEN class = {1} (p = 5.0, n = 0.0, P = 52.0, N = 48.0, weight = 1.0, pvalue = 0.03452046235551369)
IF x1 = {1} AND x2 = {0} AND x3 = {0} AND x4 = {0} AND x5 = {1} THEN class = {1} (p = 7.0, n = 0.0, P = 52.0, N = 48.0, weight = 1.0, pvalue = 0.008357585622913864)
IF x1 = {0} AND x3 = {0} THEN class = {1} (p = 14.0, n = 6.0, P = 52.0, N = 48.0, weight = 0.7, pvalue = 0.0593849499001276)
IF x2 = {0} AND x4 = {1} THEN class = {1} (p = 13.0, n = 8.0, P = 52.0, N = 48.0, weight = 0.6190476190476191, pvalue = 0.21926675708858187)
IF x1 = {1} AND x2 = {0} AND x3 = {1} AND x4 = {0} AND x5 = {0} THEN class = {1} (p = 5.0, n = 0.0, P = 52.0, N = 48.0, weight = 1.0, pvalue = 0.03452046235551369)
IF x3 = {0} THEN class = {1} (p = 32.0, n = 22.0, P = 52.0, N = 48.0, weight = 0.5925925925925926, pvalue = 0.08467798638098872)
IF x1 = {0} AND x3 = {1} AND x5 = {1} THEN class = {1} (p = 7.0, n = 6.0, P = 52.0, N = 48.0, wei

In [41]:
df = pd.DataFrame(columns=["x1", "x2", "x3","x4", "x5", "class"], index = [i for i in range(0,100)])

df["x1"] = [random.randint(0, 1) for i in range(100)]
df["x2"] = [random.randint(0, 1) for i in range(100)]
df["x3"] = [random.randint(0, 1) for i in range(100)]
df["x4"] = [random.randint(0, 1) for i in range(100)]
df["x5"] = [random.randint(0, 1) for i in range(100)]

df.loc[(df[["x1", "x2", "x3","x4", "x5"]].sum(axis=1)%2) == 0, "class"] = "1"
df = df.fillna("0")
df = df.astype("str")
df

,x1,x2,x3,x4,x5,class
0,0,0,0,0,0,1
1,0,1,1,1,0,0
2,1,0,0,1,1,0
3,0,0,0,1,1,1
4,0,1,0,0,1,1
...,...,...,...,...,...,...
95,0,0,0,1,0,0
96,1,1,1,1,1,0
97,0,1,1,1,0,0
98,1,0,0,0,0,0


In [42]:
X = df.drop(['class'], axis=1)
y = df['class']

print(f"Rules original score: {clf_simple.score(X, y)}")

print(f"Complex rules score: {clf_complex.score(X, y)}")

Rules original score: 0.62
Complex rules score: 0.92


# Complex cases 1

In [49]:
df = pd.DataFrame(columns=["a", "b", "class"], index = [i for i in range(0,100)])


a = [round(random.uniform(0, 9),2) for i in range(100)]
b = [round(random.uniform(0, 9),2)for i in range(100)]

df["a"] = a
df["b"] = b

df.loc[((df["a"] >= 7) & (df["b"] <= 3)) | ((df["a"] <=3) & (df["b"] >= 7)) ,"class"] = "1"
df = df.fillna("0")
df

,a,b,class
0,1.38,3.01,0
1,4.50,4.76,0
2,5.33,4.91,0
3,5.19,4.09,0
4,6.57,1.77,0
...,...,...,...
95,3.11,1.63,0
96,7.03,3.95,0
97,0.01,5.63,0
98,7.35,5.00,0


In [50]:
fig = px.scatter(df, x='a',
            y='b',
            color='class',
            height=560,
            width=560,
            )
fig.show()

In [51]:
X = df.drop(['class'], axis=1)
y = df['class']

clf_simple.fit(X, y)

print("Rules original")
for rule in clf_simple.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules")
clf_complex.fit(X, y)
for rule in clf_complex.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules + inners alternatives")
clf_complex_with_inners.fit(X, y)
for rule in clf_complex_with_inners.model.rules:
    print(rule, rule.stats)

Rules original
IF b = (-inf, 1.05) THEN class = {0} (p = 9.0, n = 0.0, P = 84.0, N = 16.0, weight = 1.0, pvalue = 0.19352887665444188)
IF a = (-inf, 7.13) AND b = (-inf, 6.97) THEN class = {0} (p = 57.0, n = 0.0, P = 84.0, N = 16.0, weight = 1.0, pvalue = 1.9703537163860036e-07)
IF b = <2.95, 6.97) THEN class = {0} (p = 41.0, n = 0.0, P = 84.0, N = 16.0, weight = 1.0, pvalue = 8.15187000734892e-05)
IF a = <3.05, inf) AND b = <3.07, inf) THEN class = {0} (p = 43.0, n = 0.0, P = 84.0, N = 16.0, weight = 1.0, pvalue = 4.302243493066098e-05)
IF a = (-inf, 3.52) AND b = <8.05, inf) THEN class = {1} (p = 6.0, n = 0.0, P = 16.0, N = 84.0, weight = 1.0, pvalue = 6.717825491563948e-06)
IF a = <8.39, inf) AND b = (-inf, 3.28) THEN class = {1} (p = 4.0, n = 0.0, P = 16.0, N = 84.0, weight = 1.0, pvalue = 0.0004641406703262369)
IF a = (-inf, 3.14) AND b = <6.97, inf) THEN class = {1} (p = 10.0, n = 0.0, P = 16.0, N = 84.0, weight = 1.0, pvalue = 4.626144911014731e-10)


Complex rules
IF a = <3.05,

# Complex case 2

In [52]:
df = pd.DataFrame(columns=["a", "b", "class"], index = [i for i in range(0,100)])


a = [round(random.uniform(0, 20),2) for i in range(100)]
b = [round(random.uniform(0, 9),2) for i in range(100)]

df["a"] = a
df["b"] = b

df.loc[(df["a"] <= 3) | ((df["a"] >=5) & (df["a"] <= 7) | ((df["a"] >=14))) ,"class"] = "1"
df = df.fillna("0")
df

,a,b,class
0,8.34,2.67,0
1,9.04,5.46,0
2,19.07,3.15,1
3,9.13,3.67,0
4,1.31,2.00,1
...,...,...,...
95,16.64,0.04,1
96,16.90,8.07,1
97,13.74,5.92,0
98,13.20,1.31,0


In [53]:
fig = px.scatter(df, x='a',
            y='b',
            color='class',
            height=560,
            width=560,
            )
fig.show()

In [54]:
X = df.drop(['class'], axis=1)
y = df['class']

clf_simple.fit(X, y)

print("Rules original")
for rule in clf_simple.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules")
clf_complex.fit(X, y)
for rule in clf_complex.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules + inners alternatives")
clf_complex_with_inners.fit(X, y)
for rule in clf_complex_with_inners.model.rules:
    print(rule, rule.stats)

Rules original
IF a = <7.06, 14.00) THEN class = {0} (p = 40.0, n = 0.0, P = 46.0, N = 54.0, weight = 1.0, pvalue = 6.8140982473079975e-22)
IF a = <3.30, 5.02) THEN class = {0} (p = 6.0, n = 0.0, P = 46.0, N = 54.0, weight = 1.0, pvalue = 0.007857724207425823)
IF a = <14.00, inf) THEN class = {1} (p = 26.0, n = 0.0, P = 54.0, N = 46.0, weight = 1.0, pvalue = 2.6836384478821624e-09)
IF a = (-inf, 3.30) THEN class = {1} (p = 18.0, n = 0.0, P = 54.0, N = 46.0, weight = 1.0, pvalue = 3.1608640099081543e-06)
IF a = <5.02, 7.06) THEN class = {1} (p = 10.0, n = 0.0, P = 54.0, N = 46.0, weight = 1.0, pvalue = 0.001382454382471896)


Complex rules
IF a = <7.06, 14.25) THEN class = {0} (p = 40.0, n = 0.0, P = 46.0, N = 54.0, weight = 1.0, pvalue = 6.8140982473079975e-22)
IF a = <3.30, 5.38) THEN class = {0} (p = 6.0, n = 0.0, P = 46.0, N = 54.0, weight = 1.0, pvalue = 0.007857724207425823)
IF a != <2.13, 14.25) THEN class = {1} (p = 39.0, n = 0.0, P = 54.0, N = 46.0, weight = 1.0, pvalue = 9.601

# Monk-1

In [43]:
train = pd.read_parquet("./data/monk_1/train.parquet")
test = pd.read_parquet("./data/monk_1/train.parquet")

In [44]:
train

,a1,a2,a3,a4,a5,a6,class
0,1,1,1,1,3,1,True
1,1,1,1,1,3,2,True
2,1,1,1,3,2,1,True
3,1,1,1,3,3,2,True
4,1,1,2,1,2,1,True
...,...,...,...,...,...,...,...
119,3,3,2,1,4,2,True
120,3,3,2,3,1,2,True
121,3,3,2,3,2,2,True
122,3,3,2,3,3,2,True


In [45]:
X = train.drop(['class'], axis=1)
y = train['class']

clf_simple.fit(X, y)

print("Rules original")
for rule in clf_simple.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules")
clf_complex.fit(X, y)
for rule in clf_complex.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules + inners alternatives")
clf_complex_with_inners.fit(X, y)
for rule in clf_complex_with_inners.model.rules:
    print(rule, rule.stats)

Rules original
IF a5 = {1} THEN class = {True} (p = 29.0, n = 0.0, P = 62.0, N = 62.0, weight = 1.0, pvalue = 2.4856745128420078e-11)
IF a1 = {3} AND a2 = {3} THEN class = {True} (p = 17.0, n = 0.0, P = 62.0, N = 62.0, weight = 1.0, pvalue = 2.142364554238474e-06)
IF a4 = {1} AND a6 = {1} THEN class = {True} (p = 15.0, n = 6.0, P = 62.0, N = 62.0, weight = 0.7142857142857143, pvalue = 0.02680801791894211)
IF a4 = {1} THEN class = {True} (p = 26.0, n = 16.0, P = 62.0, N = 62.0, weight = 0.6190476190476191, pvalue = 0.043548750924814826)
IF a3 = {1} AND a6 = {2} THEN class = {True} (p = 20.0, n = 14.0, P = 62.0, N = 62.0, weight = 0.5882352941176471, pvalue = 0.15710116246905934)
IF a6 = {1} THEN class = {True} (p = 29.0, n = 27.0, P = 62.0, N = 62.0, weight = 0.5178571428571429, pvalue = 0.42844949591782744)
IF a1 = {1} AND a2 = {3} AND a5 = {4} THEN class = {False} (p = 8.0, n = 0.0, P = 62.0, N = 62.0, weight = 1.0, pvalue = 0.003071215409809354)
IF a1 = {1} AND a2 = {2} AND a5 = {4} 

The Monk's Problems: Problem 1
Once upon a time, in July 1991, the monks of Corsendonk Priory were faced with a school held in their priory, namely the 2nd European Summer School on Machine Learning. After listening more than one week to a wide variety of learning algorithms, they felt rather confused: Which algorithm would be optimal? And which one to avoid? As a consequence of this dilemma, they created a simple task on which all learning algorithms ought to be compared: the three MONK's problems.

The target concept associated with the 1st Monk's problem is the binary outcome of the logical formula:
MONK-1: (a1 == a2) or (a5 == 1)

In this dataset, the original train and test sets were merged to allow other sampling procedures. However, the original train-test splits can be found as one of the OpenML tasks.

# Monk-2

In [46]:
train = pd.read_parquet("./data/monk_2/train.parquet")
test = pd.read_parquet("./data/monk_2/train.parquet")

In [47]:
train

,a1,a2,a3,a4,a5,a6,class
0,1,1,1,1,2,2,False
1,1,1,1,1,4,1,False
2,1,1,1,2,1,1,False
3,1,1,1,2,1,2,False
4,1,1,1,2,2,1,False
...,...,...,...,...,...,...,...
164,3,3,2,2,3,1,False
165,3,3,2,2,3,2,False
166,3,3,2,3,1,1,True
167,3,3,2,3,2,1,False


In [48]:
X = train.drop(['class'], axis=1)
y = train['class']

clf_simple.fit(X, y)

print("Rules original")
for rule in clf_simple.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules")
clf_complex.fit(X, y)
for rule in clf_complex.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules + inners alternatives")
clf_complex_with_inners.fit(X, y)
for rule in clf_complex_with_inners.model.rules:
    print(rule, rule.stats)

Rules original
IF a4 = {1} AND a5 = {1} AND a6 = {1} THEN class = {False} (p = 9.0, n = 0.0, P = 105.0, N = 64.0, weight = 1.0, pvalue = 0.012043080252488826)
IF a4 = {1} AND a5 = {1} THEN class = {False} (p = 14.0, n = 1.0, P = 105.0, N = 64.0, weight = 0.9333333333333333, pvalue = 0.0060842019563005865)
IF a1 = {1} AND a3 = {1} AND a4 = {1} THEN class = {False} (p = 7.0, n = 0.0, P = 105.0, N = 64.0, weight = 1.0, pvalue = 0.03304309058967097)
IF a3 = {1} AND a4 = {1} AND a6 = {1} THEN class = {False} (p = 11.0, n = 0.0, P = 105.0, N = 64.0, weight = 1.0, pvalue = 0.004317330656552595)
IF a4 = {1} AND a6 = {1} THEN class = {False} (p = 24.0, n = 4.0, P = 105.0, N = 64.0, weight = 0.8571428571428571, pvalue = 0.0032296489767082346)
IF a2 = {2} AND a3 = {2} AND a4 = {1} AND a6 = {2} THEN class = {False} (p = 5.0, n = 0.0, P = 105.0, N = 64.0, weight = 1.0, pvalue = 0.08922301996394814)
IF a2 = {1} AND a5 = {4} AND a6 = {1} THEN class = {False} (p = 9.0, n = 0.0, P = 105.0, N = 64.0, we

# Monk-3

In [49]:
train = pd.read_parquet("./data/monk_3/train.parquet")
test = pd.read_parquet("./data/monk_3/train.parquet")

In [50]:
train

,a1,a2,a3,a4,a5,a6,class
0,1,1,1,1,1,2,True
1,1,1,1,1,2,1,True
2,1,1,1,1,2,2,True
3,1,1,1,1,3,1,False
4,1,1,1,1,4,1,False
...,...,...,...,...,...,...,...
117,3,3,2,2,2,2,False
118,3,3,2,2,3,2,False
119,3,3,2,3,1,1,False
120,3,3,2,3,3,2,False


In [51]:
X = train.drop(['class'], axis=1)
y = train['class']

clf_simple.fit(X, y)

print("Rules original")
for rule in clf_simple.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules")
clf_complex.fit(X, y)
for rule in clf_complex.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules + inners alternatives")
clf_complex_with_inners.fit(X, y)
for rule in clf_complex_with_inners.model.rules:
    print(rule, rule.stats)

Rules original
IF a2 = {2} AND a5 = {1} THEN class = {True} (p = 10.0, n = 0.0, P = 60.0, N = 62.0, weight = 1.0, pvalue = 0.0005470406191995192)
IF a1 = {1} AND a2 = {2} AND a5 = {2} THEN class = {True} (p = 8.0, n = 0.0, P = 60.0, N = 62.0, weight = 1.0, pvalue = 0.0026572312430347715)
IF a2 = {2} AND a3 = {2} AND a5 = {3} THEN class = {True} (p = 5.0, n = 0.0, P = 60.0, N = 62.0, weight = 1.0, pvalue = 0.0263474580998908)
IF a2 = {2} AND a6 = {2} THEN class = {True} (p = 18.0, n = 4.0, P = 60.0, N = 62.0, weight = 0.8181818181818182, pvalue = 0.0006392617157850593)
IF a2 = {1} AND a5 = {1} THEN class = {True} (p = 12.0, n = 0.0, P = 60.0, N = 62.0, weight = 1.0, pvalue = 0.00010780642833323966)
IF a2 = {1} AND a5 = {2} THEN class = {True} (p = 9.0, n = 0.0, P = 60.0, N = 62.0, weight = 1.0, pvalue = 0.001212070391559723)
IF a2 = {1} AND a5 = {3} THEN class = {True} (p = 5.0, n = 1.0, P = 60.0, N = 62.0, weight = 0.8333333333333334, pvalue = 0.09615696246712296)
IF a3 = {1} THEN clas

# Trains

In [52]:
train = pd.read_csv("./data/trains/trains-transformed.csv")

In [53]:
train

,Number_of_cars,Number_of_different_loads,num_wheels_car_2,length_car_2,shape_car_2,num_loads_car_2,load_shape_car_2,num_wheels_car_3,length_car_3,shape_car_3,...,Rectangle_next_to_triangle,Rectangle_next_to_hexagon,Rectangle_next_to_circle,Triangle_next_to_triangle,Triangle_next_to_hexagon,Triangle_next_to_circle,Hexagon_next_to_hexagon,Hexagon_next_to_circle,Circle_next_to_circle,class
0,5,4,2,long,openrect,3,rectanglod,2,short,slopetop,...,1,0,0,0,1,0,0,1,0,east
1,4,3,2,short,ushaped,1,trianglod,2,short,opentrap,...,1,0,1,0,0,0,0,0,0,east
2,4,2,2,short,openrect,1,circlelod,2,short,hexagon,...,0,0,0,1,0,1,0,0,0,east
3,5,2,2,short,opentrap,1,trianglod,2,short,dblopnrect,...,1,0,0,1,0,0,0,0,0,east
4,4,3,2,short,dblopnrect,1,trianglod,3,long,closedrect,...,1,0,1,0,0,0,0,0,0,east
5,3,2,2,long,closedrect,3,circlelod,2,short,openrect,...,0,0,0,0,0,1,0,0,0,west
6,4,2,2,short,dblopnrect,1,circlelod,2,short,ushaped,...,0,0,0,0,0,1,0,0,0,west
7,3,2,3,long,closedrect,1,rectanglod,2,short,ushaped,...,0,0,1,0,0,0,0,0,0,west
8,5,2,2,short,opentrap,1,circlelod,2,long,jaggedtop,...,0,0,1,0,0,0,0,0,0,west
9,3,1,2,short,ushaped,1,rectanglod,2,long,openrect,...,0,0,0,0,0,0,0,0,0,west


In [54]:
train.dtypes

Number_of_cars                  int64
Number_of_different_loads       int64
num_wheels_car_2                int64
length_car_2                   object
shape_car_2                    object
num_loads_car_2                 int64
load_shape_car_2               object
num_wheels_car_3                int64
length_car_3                   object
shape_car_3                    object
num_loads_car_3                 int64
load_shape_car_3               object
num_wheels_car_4               object
length_car_4                   object
shape_car_4                    object
num_loads_car_4                object
load_shape_car_4               object
num_wheels_car_5               object
length_car_5                   object
shape_car_5                    object
num_loads_car_5                object
load_shape_car_5               object
Rectangle_next_to_rectangle     int64
Rectangle_next_to_triangle      int64
Rectangle_next_to_hexagon       int64
Rectangle_next_to_circle        int64
Triangle_nex

In [55]:
train= train.astype('str')

In [56]:
train.dtypes

Number_of_cars                 object
Number_of_different_loads      object
num_wheels_car_2               object
length_car_2                   object
shape_car_2                    object
num_loads_car_2                object
load_shape_car_2               object
num_wheels_car_3               object
length_car_3                   object
shape_car_3                    object
num_loads_car_3                object
load_shape_car_3               object
num_wheels_car_4               object
length_car_4                   object
shape_car_4                    object
num_loads_car_4                object
load_shape_car_4               object
num_wheels_car_5               object
length_car_5                   object
shape_car_5                    object
num_loads_car_5                object
load_shape_car_5               object
Rectangle_next_to_rectangle    object
Rectangle_next_to_triangle     object
Rectangle_next_to_hexagon      object
Rectangle_next_to_circle       object
Triangle_nex

In [57]:
X = train.drop(['class'], axis=1)
y = train['class']

clf_simple.fit(X, y)

print("Rules original")
for rule in clf_simple.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules")
clf_complex.fit(X, y)
for rule in clf_complex.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules + inners alternatives")
clf_complex_with_inners.fit(X, y)
for rule in clf_complex_with_inners.model.rules:
    print(rule, rule.stats)

Rules original
IF Rectangle_next_to_triangle = {1} THEN class = {east} (p = 4.0, n = 0.0, P = 5.0, N = 5.0, weight = 1.0, pvalue = 0.023809523809523808)
IF shape_car_4 = {closedrect} THEN class = {east} (p = 3.0, n = 0.0, P = 5.0, N = 5.0, weight = 1.0, pvalue = 0.08333333333333333)
IF load_shape_car_4 = {-} THEN class = {west} (p = 4.0, n = 0.0, P = 5.0, N = 5.0, weight = 1.0, pvalue = 0.023809523809523808)
IF shape_car_3 = {jaggedtop} THEN class = {west} (p = 1.0, n = 0.0, P = 5.0, N = 5.0, weight = 1.0, pvalue = 0.49999999999999994)


Complex rules
IF Triangle_next_to_triangle != Rectangle_next_to_triangle THEN class = {east} (p = 4.0, n = 0.0, P = 5.0, N = 5.0, weight = 1.0, pvalue = 0.023809523809523808)
IF Rectangle_next_to_triangle != {0} THEN class = {east} (p = 4.0, n = 0.0, P = 5.0, N = 5.0, weight = 1.0, pvalue = 0.023809523809523808)
IF Triangle_next_to_triangle = Hexagon_next_to_circle = Rectangle_next_to_triangle THEN class = {west} (p = 5.0, n = 0.0, P = 5.0, N = 5.0, we

# balance-scale

In [58]:
train = pd.read_parquet("./data/balance-scale.parquet")

In [59]:
train

,left-weight,left-distance,right-weight,right-distance,class
0,1.0,1.0,1.0,1.0,B
1,1.0,1.0,1.0,2.0,R
2,1.0,1.0,1.0,3.0,R
3,1.0,1.0,1.0,4.0,R
4,1.0,1.0,1.0,5.0,R
...,...,...,...,...,...
620,5.0,5.0,5.0,1.0,L
621,5.0,5.0,5.0,2.0,L
622,5.0,5.0,5.0,3.0,L
623,5.0,5.0,5.0,4.0,L


In [60]:
X = train.drop(['class'], axis=1)
y = train['class']

clf_simple.fit(X, y)

print("Rules original")
for rule in clf_simple.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules")
clf_complex.fit(X, y)
for rule in clf_complex.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules + inners alternatives")
clf_complex_with_inners.fit(X, y)
for rule in clf_complex_with_inners.model.rules:
    print(rule, rule.stats)

Rules original
IF right-distance = (-inf, 2.50) AND left-weight = (-inf, 2.50) AND left-distance = (-inf, 2.50) AND right-weight = (-inf, 2.50) THEN class = {B} (p = 6.0, n = 10.0, P = 49.0, N = 576.0, weight = 0.375, pvalue = 0.0007499248952481466)
IF right-distance = (-inf, 2.50) AND left-weight = <1.50, 4.50) AND left-distance = (-inf, 2.50) AND right-weight = <1.50, 4.50) THEN class = {B} (p = 8.0, n = 28.0, P = 49.0, N = 576.0, weight = 0.2222222222222222, pvalue = 0.004434784254422619)
IF left-weight = <1.50, 4.50) AND right-distance = <1.50, 4.50) AND left-distance = (-inf, 2.50) AND right-weight = (-inf, 2.50) THEN class = {B} (p = 8.0, n = 28.0, P = 49.0, N = 576.0, weight = 0.2222222222222222, pvalue = 0.004434784254422619)
IF left-weight = (-inf, 2.50) AND right-distance = (-inf, 2.50) AND left-distance = <1.50, 4.50) AND right-weight = <1.50, 4.50) THEN class = {B} (p = 8.0, n = 28.0, P = 49.0, N = 576.0, weight = 0.2222222222222222, pvalue = 0.004434784254422619)
IF left-w

# flag

In [61]:
train = pd.read_parquet("./data/flag.parquet")

In [62]:
train

,landmass,botright,area,population,language,religion,bars,stripes,colours,red,...,saltires,quarters,sunstars,crescent,triangle,icon,animate,text,topleft,class
0,5,green,648.0,16.0,10,2,0.0,3.0,5.0,1,...,0.0,0.0,1.0,0,0,1,0,0,black,1
1,3,red,29.0,3.0,6,6,0.0,0.0,3.0,1,...,0.0,0.0,1.0,0,0,0,1,0,red,1
2,4,white,2388.0,20.0,8,2,2.0,0.0,3.0,1,...,0.0,0.0,1.0,1,0,0,0,0,green,1
3,6,red,0.0,0.0,1,1,0.0,0.0,5.0,1,...,0.0,0.0,0.0,0,1,1,1,0,blue,3
4,3,red,0.0,0.0,6,0,3.0,0.0,3.0,1,...,0.0,0.0,0.0,0,0,0,0,0,blue,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,6,red,3.0,0.0,1,1,0.0,0.0,3.0,1,...,0.0,1.0,5.0,0,0,0,0,0,blue,3
190,3,red,256.0,22.0,6,6,0.0,3.0,4.0,1,...,0.0,0.0,1.0,0,0,0,0,0,blue,1
191,4,green,905.0,28.0,10,5,0.0,0.0,4.0,1,...,0.0,0.0,0.0,0,0,1,1,0,green,2
192,4,brown,753.0,6.0,10,5,3.0,0.0,4.0,1,...,0.0,0.0,0.0,0,0,0,1,0,green,2


In [63]:
X = train.drop(['class'], axis=1)
y = train['class']

clf_simple.fit(X, y)

print("Rules original")
for rule in clf_simple.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules")
clf_complex.fit(X, y)
for rule in clf_complex.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules + inners alternatives")
clf_complex_with_inners.fit(X, y)
for rule in clf_complex_with_inners.model.rules:
    print(rule, rule.stats)

Rules original
IF landmass = {5} THEN class = {1} (p = 39.0, n = 0.0, P = 91.0, N = 103.0, weight = 1.0, pvalue = 6.039529644312067e-16)
IF language = {4} THEN class = {1} (p = 6.0, n = 0.0, P = 91.0, N = 103.0, weight = 1.0, pvalue = 0.009733415084710487)
IF stripes = (-inf, 4) AND religion = {6} THEN class = {1} (p = 13.0, n = 0.0, P = 91.0, N = 103.0, weight = 1.0, pvalue = 3.2661270084870434e-05)
IF language = {8} AND population = <2.50, inf) THEN class = {1} (p = 10.0, n = 1.0, P = 91.0, N = 103.0, weight = 0.9090909090909091, pvalue = 0.0025797174195113966)
IF area = (-inf, 477.50) AND landmass = {3} AND colours = (-inf, 2.50) THEN class = {1} (p = 9.0, n = 0.0, P = 91.0, N = 103.0, weight = 1.0, pvalue = 0.0008821230055869394)
IF red = {1} AND landmass = {3} AND bars = <1, inf) THEN class = {1} (p = 7.0, n = 0.0, P = 91.0, N = 103.0, weight = 1.0, pvalue = 0.004400746181916965)
IF area = (-inf, 935) AND white = {1} AND crosses = (-inf, 0.50) AND population = <12.50, inf) AND col

# mushroom

In [64]:
train = pd.read_parquet("./data/mushroom.parquet")

In [65]:
train

,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class
0,x,s,n,t,p,f,c,n,k,e,...,w,w,p,w,o,p,k,s,u,p
1,x,s,y,t,a,f,c,b,k,e,...,w,w,p,w,o,p,n,n,g,e
2,b,s,w,t,l,f,c,b,n,e,...,w,w,p,w,o,p,n,n,m,e
3,x,y,w,t,p,f,c,n,n,e,...,w,w,p,w,o,p,k,s,u,p
4,x,s,g,f,n,f,w,b,k,t,...,w,w,p,w,o,e,n,a,g,e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,k,s,n,f,n,a,c,b,y,e,...,o,o,p,o,o,p,b,c,l,e
8120,x,s,n,f,n,a,c,b,y,e,...,o,o,p,n,o,p,b,v,l,e
8121,f,s,n,f,n,a,c,b,n,e,...,o,o,p,o,o,p,b,c,l,e
8122,k,y,n,f,y,f,c,n,b,t,...,w,w,p,w,o,e,w,v,l,p


In [66]:
X = train.drop(['class'], axis=1)
y = train['class']

clf_simple.fit(X, y)

print("Rules original")
for rule in clf_simple.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules")
clf_complex.fit(X, y)
for rule in clf_complex.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules + inners alternatives")
clf_complex_with_inners.fit(X, y)
for rule in clf_complex_with_inners.model.rules:
    print(rule, rule.stats)

Rules original
IF odor = {f} THEN class = {p} (p = 2160.0, n = 0.0, P = 3916.0, N = 4208.0, weight = 1.0, pvalue = 0.0)
IF gill-color = {b} THEN class = {p} (p = 1728.0, n = 0.0, P = 3916.0, N = 4208.0, weight = 1.0, pvalue = 0.0)
IF odor = {p} THEN class = {p} (p = 256.0, n = 0.0, P = 3916.0, N = 4208.0, weight = 1.0, pvalue = 8.488489938344628e-84)
IF odor = {c} THEN class = {p} (p = 192.0, n = 0.0, P = 3916.0, N = 4208.0, weight = 1.0, pvalue = 1.1762049276390926e-62)
IF spore-print-color = {r} THEN class = {p} (p = 72.0, n = 0.0, P = 3916.0, N = 4208.0, weight = 1.0, pvalue = 1.0791281224127857e-23)
IF odor = {m} THEN class = {p} (p = 36.0, n = 0.0, P = 3916.0, N = 4208.0, weight = 1.0, pvalue = 3.583164811701358e-12)
IF stalk-color-below-ring = {y} THEN class = {p} (p = 24.0, n = 0.0, P = 3916.0, N = 4208.0, weight = 1.0, pvalue = 2.3869571934119588e-08)
IF stalk-surface-above-ring = {k} AND gill-spacing = {c} THEN class = {p} (p = 2228.0, n = 0.0, P = 3916.0, N = 4208.0, weight =

# tic-tac-toe

In [67]:
train = pd.read_parquet("./data/tic-tac-toe.parquet")

In [68]:
train

,top-left-square,top-middle-square,top-right-square,middle-left-square,middle-middle-square,middle-right-square,bottom-left-square,bottom-middle-square,bottom-right-square,class
0,x,x,x,x,o,o,x,o,o,positive
1,x,x,x,x,o,o,o,x,o,positive
2,x,x,x,x,o,o,o,o,x,positive
3,x,x,x,x,o,o,o,b,b,positive
4,x,x,x,x,o,o,b,o,b,positive
...,...,...,...,...,...,...,...,...,...,...
953,o,x,x,x,o,o,o,x,x,negative
954,o,x,o,x,x,o,x,o,x,negative
955,o,x,o,x,o,x,x,o,x,negative
956,o,x,o,o,x,x,x,o,x,negative


In [69]:
X = train.drop(['class'], axis=1)
y = train['class']

clf_simple.fit(X, y)

print("Rules original")
for rule in clf_simple.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules")
clf_complex.fit(X, y)
for rule in clf_complex.model.rules:
    print(rule, rule.stats)

print("\n\nComplex rules + inners alternatives")
clf_complex_with_inners.fit(X, y)
for rule in clf_complex_with_inners.model.rules:
    print(rule, rule.stats)

Rules original
IF middle-middle-square = {x} AND bottom-left-square = {b} AND top-right-square = {b} THEN class = {positive} (p = 30.0, n = 0.0, P = 626.0, N = 332.0, weight = 1.0, pvalue = 2.232953092174273e-06)
IF middle-middle-square = {x} AND bottom-left-square = {b} AND top-right-square = {x} THEN class = {positive} (p = 20.0, n = 0.0, P = 626.0, N = 332.0, weight = 1.0, pvalue = 0.00018100703894850716)
IF middle-middle-square = {x} AND bottom-middle-square = {o} AND bottom-right-square = {x} AND bottom-left-square = {b} THEN class = {positive} (p = 22.0, n = 0.0, P = 626.0, N = 332.0, weight = 1.0, pvalue = 7.550592519529088e-05)
IF middle-middle-square = {x} AND bottom-middle-square = {o} AND bottom-left-square = {b} AND middle-right-square = {x} THEN class = {positive} (p = 22.0, n = 0.0, P = 626.0, N = 332.0, weight = 1.0, pvalue = 7.550592519529088e-05)
IF middle-middle-square = {x} AND top-left-square = {x} AND middle-left-square = {o} AND bottom-left-square = {b} THEN class